# Project 2: Segmentation

### What to Submit
Submit this iPython Notebook--containing all your code for the programming exercises below--on [learning suite](https://learningsuite.byu.edu/).

Your notebook file should produce the relevant plots and also provide a short write-up with answers to the questions below.

Please also fill in here the time that each part took you:
* 1. Setting up PyMaxFlow: <span style="color:red;">FILL IN TIME</span>
* 2. Getting your first successful segmentation: <span style="color:red;">FILL IN TIME</span>
* 3. Adjusting parameters (e.g. $\lambda$, $\sigma$) and so forth, to get good results: <span style="color:red;">FILL IN TIME</span>
* 4. Completing the write-up: <span style="color:red;">FILL IN TIME</span>

Note that there are two folders within the project.  We have provided some images for you to use in testing your implementation, in the `provided images` folder along with their ground-truth segmentations to compare your results to.  Along with these, we want you to provide 2-4 additional images that you select on which you show your results.  <i>Please use the `provided_images` in that path and place any others in the `user_data` folder, and load all of the images (or user input point location files) via the approapriate relative path.  We will drop your notebook file and your `user_data` folder into our folder (which will have the `provided_images` already) and then run your notebook.</i>




## Preparation:
So that you can focus on the elements of the cost function ( the link weights), you may use a existing implementation of the actual min-cut algorithm itself.  You set up the graph, but it will take care of finding the minimum cut.

For this assignment we will be using a python library called PyMaxFlow.  This library is a python wrapper around the original C++ implementation of the min-cut code from [Vladimir Kolmogorov](http://pub.ist.ac.at/%7Evnk/software.html) (who has co-authored several papers on this subject).    

Note: For windows users, you will need the Visual C++ compiler in order for PyMaxFlow to work.  If you already have Visual Studio, this shouldn't be a problem but if you just want the compiler without Visual Studio, you can download [Build Tools For Visual Studio 2017](https://visualstudio.microsoft.com/downloads/#build-tools-for-visual-studio-2017). Once you have access to the Visual C++ compiler look at the next paragraph for PyMaxFlow installation.

PyMaxFlow requires Cython, which should come standard in your anaconda environment but the command to install that will also be included.  To install PyMaxFlow enter the following commands replacing "YourEnvironmentName" with the name of your anaconda environment.
~~~
conda activate YourEnvironemntName
conda install cython
pip install pymaxflow
~~~
Once PyMaxFlow is installed, to understand how to use the library, there is a great [tutorial page](http://pmneila.github.io/PyMaxflow/tutorial.html) that shows how to get started with some simple examples.  Do the "first example" and perhaps the "binary image restoration" as well.

In [2]:
import matplotlib.pyplot as plt
import maxflow
import numpy as np
from scipy import stats
import guiseg
import cv2
from sklearn.neighbors import KernelDensity



## Annotations:
Graph cut segmentation is an interactive algorithm requiring the user to provide foreground and background seeds.  Provided is a python file that will open a gui and allow you to annotate the image.  This gui is optional and will require additional packages to be installed into your environment.  To install the packages open a terminal and enter the following commands:
~~~
conda activate YourEnvironmentName
conda install scikit-image pillow
~~~
You can use the gui in the following way:
```python
import guiseg
fore, back = guiseg.get_fore_back(image)
image[fore]  # the foreground seeds
image[back]  # the background seeds
```

For the `guiseg` routine to run, I also found it necssary to install PIL ImageTk (For me, it was `sudo apt install python3-pil.imagetk` but it will be different for Conda).

In [3]:

#imageBGR = cv2.imread('provided_images/simplecircle.png');
# imageBGR = cv2.imread('provided_images/banana.png');
# image = imageBGR[:,:,::-1]; # I reverse the BGR from OpenCV to BGR
#
# # When the GUI pops up, you can pick either the "Foreground" or "Background" buttons to
# # select pixels to be respective seeds.  Once you're finished, click "Return"
# fore, back = guiseg.get_fore_back(image)
# print(image[fore][:5])  # foreground seeds (RGB values for all pixels drawn on, but only showing 5)
# print(image[back][:5])  # background seeds (RGB values for all pixels drawn on, but only showing 5)



## Graph Cut:
Your code should read in an image and a set of seed pixels and use graph-cut segmentation to segment the image.

You will need to calculate the costs for the t-links (region terms) and the n-links (boundary terms). See the book, the notes/slides, or published papers in this area for ideas of how to define these.  Remember that the t-link weights to a particular terminal (foreground or background) should be large if that pixel looks a lot like the respective foreground/background seeds. The n-link weights should be large if the two neighboring pixels are similar.

Here is [the original paper on graph-cut segmentation](http://www.csd.uwo.ca/~yuri/Papers/iccv01.pdf), which might help with some ideas, but you should look at the literature to see what other costs functions / link weights others have used.

Once the graph is built, use the min-cut algorithm to partition the graph into nodes connected to the foreground node or to the background node, then use these as the resulting labels for the segmentation. Display this result graphically in some fashion overlaid on the input image.  It is best to start with simple images whose foreground and background colors are pretty different and for ones where the edges are pretty clear.  Graph-cut segmentation struggles sometimes with long, thin structures, so you should avoid these types of images early on.



In [4]:
    fKDE = KernelDensity(bandwidth=1.0, kernel='gaussian').fit([[1]])
    bKDE = KernelDensity(bandwidth=1.0, kernel='gaussian').fit([[0]])
    fF = np.exp(fKDE.score([[1]]))
    fB = np.exp(bKDE.score([[1]]))
    bF = fKDE.score([[0]])
    bB = bKDE.score([[0]])

In [5]:
def imgToNode(r,c, iW):
    return r*iW + c
def nodeToImg(nIndex, iW):
    c = nIndex % iW
    r = np.round((nIndex - c) / iW)
    return r,c

In [1]:
# Implementation Here (feel free to add additional cells of course)
def calcNLink(sourcePixel, img, graph, nodes):
    sourceRGB = img[sourcePixel[0],sourcePixel[1],:].astype('int')
    sigma = 50 #???
    if sourcePixel[0] + 1 < img.shape[0]:
        rightRGB = img[sourcePixel[0]+1, sourcePixel[1],:].astype('int')
        weight = (rightRGB[0]-sourceRGB[0])**2+(rightRGB[1]-sourceRGB[1])**2+(rightRGB[2]-sourceRGB[2])**2
        weight = np.exp((np.sqrt(weight) * (-1/sigma**2)))
        sNode = nodes[imgToNode(sourcePixel[0], sourcePixel[1], img.shape[1])]
        rNode = nodes[imgToNode(sourcePixel[0] + 1, sourcePixel[1], img.shape[1])]
        graph.add_edge(sNode, rNode, weight, weight)
    if sourcePixel[1] + 1 < img.shape[1]:
        botRGB = img[sourcePixel[0], sourcePixel[1]+1,:].astype('int')
        weight = (botRGB[0]-sourceRGB[0])**2+(botRGB[1]-sourceRGB[1])**2+(botRGB[2]-sourceRGB[2])**2
        weight = np.exp((np.sqrt(weight) * (-1/sigma**2)))
        sNode = nodes[imgToNode(sourcePixel[0], sourcePixel[1], img.shape[1])]
        bNode = nodes[imgToNode(sourcePixel[0], sourcePixel[1]+1, img.shape[1])]
        graph.add_edge(sNode, bNode, weight, weight)
    return graph

def calcTLink(sourcePixel, img, graph, nodes):
    nodeIndex = imgToNode(sourcePixel[0], sourcePixel[1], img.shape[1])
    graph.add_tedge(nodes[nodeIndex], 1, 1)
    return graph

def segmentaiton(img, fore, back):
    rSize, cSize, _ = img.shape
    graph = maxflow.Graph[float](rSize * cSize, rSize*(cSize-1) + (rSize-1)*(cSize))
    nodes = graph.add_nodes(rSize * cSize)

    fData = img[fore]
    bData = img[back]
    fKDE = KernelDensity(bandwidth=.5, kernel='gaussian').fit(fData)
    bKDE = KernelDensity(bandwidth=.5, kernel='gaussian').fit(bData)
    fF = np.log(np.exp(fKDE.score([img[130,130]]))) * -1
    fB = np.log(np.exp(bKDE.score([img[130,130]]))) * -1
    bF = fKDE.score([img[0,0]])
    bB = bKDE.score([img[0,0]])




    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            graph = calcNLink((r,c), img, graph, nodes)
            graph = calcTLink((r,c), img, graph, nodes)

    flow = graph.maxflow()

    segImg = np.copy(img)
    segImg = segImg[:,:,0]

    for r in range(segImg.shape[0]):
        for c in range(segImg.shape[1]):
            val = graph.get_segment(nodes[imgToNode(r,c,segImg.shape[1])])
            if val:
                segImg[r,c] = 250
            else:
                segImg[r,c] = 0
    return segImg

imageBGR = cv2.imread('provided_images/simplecircle.png');
image = imageBGR[:,:,::-1]

fore, back = guiseg.get_fore_back(image)
segImg = segmentaiton(image, fore, back)
plt.imshow(segImg, cmap='gray', vmin=0, vmax=255);plt.title("Segmented Image");plt.show()

NameError: name 'cv2' is not defined

In [1]:
# Generate results Here (again, add additional cells to your heart's content)
imageBGR = cv2.imread('provided_images/simplecircle.png');
image = imageBGR[:,:,::-1]

fore, back = guiseg.get_fore_back(image)
segImg = segmentaiton(image, fore, back)
plt.imshow(segImg, cmap='gray');plt.title("Segmented Image");plt.show()

NameError: name 'segmentaiton' is not defined


## Grading / Rubric
Points for this assigment will be assigned as follows (100 points total):
* [20 pts] Code that correctly generates the graph network structure (nodes, n-links, t-links).
* [10 pts] Code that produces the boundary term $B(p,q)$ used for n-links.
* [10 pts] Code that produces the region term of the cost $R(p,A)$ used for the t-links.  Remember that you have t-links per pixel, one with cost determined by matching $p$ with the foreground appearance distribution, the other determined relative to the background distribution.  You may use the [sk-learn implementation](https://scikit-learn.org/stable/modules/density.html#kernel-density-estimation) of Kernel Density Estimation.  However you will receive 10 extra points if you implement it yourself.
* [20 pts] Implementing the graph-cut with `pymaxflow` and finding the optimal solution for the input graph.
* [10 pts] Displaying Results in the following format (for each input image you'll show the following 3-4 result images):
   1. Original Image.
   2. Tri-map of what was selected by the user (white for foreground, black for background, gray for unknown).  This can be overlaid on top of a faint copy of the image for context if desired.
   3. Final segmentation.  Again you can overlay it on a faint copy of the original for context.
   4. On the <i>provided images</i> please show a comparison of your resulting segmentaiton with the ground truth.
* [20 pts] Good (certainly not perfect, some of them are challenging, but decent/reasonable) results on the 4 provided images (banana, llama, penguin, teddy).  Each image will receive up to 5 points.
* [10 pts] Demonstrating your algorithm on 2-4 additional images.  At least one of the images should be somewhat easy, one should be somewhat challenging -- expalin why you think they're respectively easy/challenging.


## Write-up:
Provide an explanation for the following items:
* Describe how you determinied/computed the n-link and t-link weights.
* What kinds of image does graph cut segmentation work well for? What kinds of images do you find it struggles with?
* What did you learn from the project?
* What if any suggestions do you have for improving it (for future students)?

<span style="color:red;">WRITE-UP HERE</span>